In [1]:
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import torch
import requests
import time

In [2]:
PICKLES_ADDRESS =  '../data/pickles/'
SBIR_ADDRESS = '../data/sbir_dataset/'
PUBMED_ADDRESS = '../data/pubmed_dataset/'

In [3]:
def gpu_info():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
    print('GPUs available:', torch.cuda.device_count())
    print('current GPU number: ', torch.cuda.current_device())
    print('GPU name: ', torch.cuda.get_device_name(device))
    print('GPU capability: ', torch.cuda.get_device_capability(device))
    print('GPU memory: ', torch.cuda.get_device_properties(device).total_memory)
    print('GPU memory allocated: ', torch.cuda.memory_allocated(device))
    print('GPU memory cached: ', torch.cuda.memory_cached(device))
    print('GPU memory reserved: ', torch.cuda.memory_reserved(device))
    print('GPU memory free: ', torch.cuda.memory_reserved(device) - torch.cuda.memory_allocated(device))


def ram_info():
    !free -h

gpu_info()  
ram_info()

GPUs available: 3
current GPU number:  0
GPU name:  NVIDIA RTX 6000 Ada Generation
GPU capability:  (8, 9)
GPU memory:  51010207744
GPU memory allocated:  0
GPU memory cached:  0
GPU memory reserved:  0
GPU memory free:  0
               total        used        free      shared  buff/cache   available
Mem:           503Gi       164Gi        54Gi       206Mi       284Gi       334Gi
Swap:          3.0Ti          0B       3.0Ti


/usr/lib/python3/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [4]:
pubmed_df = pd.read_csv(PUBMED_ADDRESS + 'out.csv')
pubmed_df.head()

,pmid,pub_date,pub_title,journal_title,doi,abstract,relative_citation_ratio,citation_count,citations_per_year,num_major_topics,major_topics,num_minor_topics,minor_topics,concept_list,yearly_citation_counts
0,28867731,2017-01-01,FOXA1 Induces E-Cadherin Expression at the Pro...,Biological & pharmaceutical bulletin,10.1248/bpb.b17-00307,Epithelial-to-mesenchymal transition (EMT) is ...,0.88,18,3.600000,NaN,NaN,15.0,"[""Antigens, CD"",""Breast Neoplasms"",""Cadherins""...","[""SFN protein, human"",""Hepatocyte Nuclear Fact...","{2019:5,2020:3,2021:1,2022:2}"
1,23611783,2013-04-20,Identification and characterization of a novel...,Biochemical and biophysical research communica...,10.1016/j.bbrc.2013.04.022,"In plants, flavonoids have been shown to be su...",0.55,15,1.666670,NaN,NaN,17.0,"[""Amino Acid Sequence"",""Arabidopsis"",""Arabidop...","[""Differential quality"",""Characterization"",""Bo...","{2013:1,2014:1,2015:1,2018:1,2019:2,2022:1}"
2,31440131,2019-08-07,The Verbal Interaction Social Threat Task: A N...,Frontiers in neuroscience,10.3389/fnins.2019.00830,"a priori In recent years, digital communicatio...",0.31,2,0.666667,NaN,NaN,NaN,NaN,"[""Neural"",""Societies"",""conversation"",""Moderate...",{2021:1}
3,30551143,2019-02-28,Functionality and cross-regulation of the four...,Nucleic acids research,10.1093/nar/gky1256,Toxin-antitoxin (TA) systems are ubiquitous am...,1.37,14,4.666670,NaN,NaN,10.0,"[""Bacterial Proteins"",""Drug Resistance, Microb...","[""Antibiotic Resistance, Microbial"",""Cross-Tal...","{2019:3,2020:2,2021:5,2022:2}"
4,30229639,2018-09-18,Non-familial porphyria cutanea tarda: a rare d...,Giornale italiano di dermatologia e venereolog...,10.23736/S0392-0488.18.06052-2,NaN,0.00,0,0.000000,NaN,NaN,5.0,"[""Female"",""Humans"",""Middle Aged"",""Porphyria Cu...",NaN,NaN


In [5]:
pubmed_df.tail()

,pmid,pub_date,pub_title,journal_title,doi,abstract,relative_citation_ratio,citation_count,citations_per_year,num_major_topics,major_topics,num_minor_topics,minor_topics,concept_list,yearly_citation_counts
13517348,26544177,2015-11-06,Determinants of Visceral Leishmaniasis: A Case...,PLoS neglected tropical diseases,10.1371/journal.pntd.0004187,Improving knowledge on local determinants of v...,1.00,13,1.85714,NaN,NaN,22.0,"[""Adolescent"",""Adult"",""Age Factors"",""Aged"",""Ag...","[""Improving (qualifier value)"",""Use of"",""Discu...","{2016:1,2017:1,2018:4,2019:1,2020:2}"
13517349,33027166,2020-11-01,What Factors Increase Revision Surgery Risk Wh...,Journal of orthopaedic trauma,10.1097/BOT.0000000000001936,HEALTH was a randomized controlled trial compa...,0.23,1,0.50000,3.0,"[""Arthroplasty, Replacement, Hip"",""Femoral Nec...",6.0,"[""Aged"",""Bone Cements"",""Female"",""Humans"",""Reop...","[""Surgery specialty"",""Increased"",""month"",""Pati...",NaN
13517350,22809828,2012-07-17,Seed shape in model legumes: approximation by ...,Journal of plant physiology,10.1016/j.jplph.2012.05.019,Seed shape in the model legumes Lotus japonicu...,0.23,7,0.70000,1.0,"[""Models, Biological""]",9.0,"[""Analysis of Variance"",""Desiccation"",""Ethylen...","[""Disease"",""Morphology"",""Mutant"",""Study models...","{2016:1,2020:1,2021:2,2022:2}"
13517351,32457164,2020-05-26,Imperfect transparency and camouflage in glass...,Proceedings of the National Academy of Science...,10.1073/pnas.1919417117,Camouflage patterns prevent detection and/or r...,1.79,9,4.50000,NaN,NaN,12.0,"[""Adaptation, Biological"",""Animals"",""Anura"",""C...","[""Biological Camouflage"",""Increasing"",""Color"",...","{2020:1,2021:2}"
13517352,25390051,2014-11-13,Antibiotic resistant Shigella is a major cause...,Journal of infection in developing countries,10.3855/jidc.4396,Diarrhoea remains a major cause of illness in ...,0.49,11,1.37500,1.0,"[""Drug Resistance, Bacterial""]",22.0,"[""Adolescent"",""Adult"",""Aged"",""Aged, 80 and ove...","[""Papua New Guinea"",""Burden"",""Established"",""Di...","{2014:1,2017:1,2019:2,2020:1,2021:1}"


In [9]:
import re
import numpy as np

all_unique_mesh_topics = []

# Iterate over rows in the DataFrame
for index, row in tqdm(pubmed_df.iterrows()):
    
    minor, majors = [], []

    if str(row['minor_topics']) != 'nan':
        minors = re.findall(r'"(.*?)"', str(row['minor_topics']))
    if str(row['major_topics']) != 'nan':
        majors = re.findall(r'"(.*?)"', str(row['major_topics']))
        
    unique_mesh_topics = ';'.join(list(set(minors + majors)))
    all_unique_mesh_topics.append(unique_mesh_topics)

    if index % 1000000 == 0:
        with open(PICKLES_ADDRESS + 'all_unique_mesh_topics.pkl', 'wb') as handle:
            pickle.dump(all_unique_mesh_topics, handle)
        print('all_unique_mesh_topics.pkl is saved.')

print(len(all_unique_mesh_topics))
with open(PICKLES_ADDRESS + 'all_unique_mesh_topics.pkl', 'wb') as handle:
    pickle.dump(all_unique_mesh_topics, handle)
print('all_unique_mesh_topics.pkl is saved.')

4626it [00:03, 2029.85it/s]

all_unique_mesh_topics.pkl is saved.


1004766it [00:43, 15791.84it/s]

all_unique_mesh_topics.pkl is saved.


2002495it [01:22, 8295.60it/s] 

...

all_unique_mesh_topics.pkl is saved.


13004367it [09:12, 2089.35it/s] 

all_unique_mesh_topics.pkl is saved.


13517353it [09:33, 23589.24it/s]


13517353
all_unique_mesh_topics.pkl is saved.


In [6]:
with open(PICKLES_ADDRESS + 'all_unique_mesh_topics.pkl', 'rb') as handle:
    all_unique_mesh_topics = pickle.load(handle)

all_unique_mesh_topics[:10]

['Hepatocyte Nuclear Factor 3-alpha;Estrogen Receptor alpha;Plasmids;Snail Family Transcription Factors;RNA, Messenger;Cadherins;Epithelial Cells;Humans;Gene Knockdown Techniques;MCF-7 Cells;Female;Breast Neoplasms;Protein Processing, Post-Translational;Antigens, CD;Epithelial-Mesenchymal Transition',
 'Kaempferols;Multigene Family;Arabidopsis Proteins;Flavonoids;Sequence Homology, Amino Acid;Phylogeny;Kinetics;Arylsulfotransferase;Isoenzymes;Sulfates;Molecular Structure;Cloning, Molecular;Molecular Sequence Data;Sulfotransferases;Amino Acid Sequence;Arabidopsis;Substrate Specificity',
 'Kaempferols;Multigene Family;Arabidopsis Proteins;Flavonoids;Sequence Homology, Amino Acid;Phylogeny;Kinetics;Arylsulfotransferase;Isoenzymes;Sulfates;Molecular Structure;Cloning, Molecular;Molecular Sequence Data;Sulfotransferases;Amino Acid Sequence;Arabidopsis;Substrate Specificity',
 'Staphylococcal Infections;Drug Resistance, Microbial;Staphylococcus aureus;Humans;Bacterial Proteins;Gene Expressio

In [7]:
len(all_unique_mesh_topics)

13517353

In [8]:
pubmed_df['unique_mesh_topics'] = all_unique_mesh_topics
pubmed_df.head()

,pmid,pub_date,pub_title,journal_title,doi,abstract,relative_citation_ratio,citation_count,citations_per_year,num_major_topics,major_topics,num_minor_topics,minor_topics,concept_list,yearly_citation_counts,unique_mesh_topics
0,28867731,2017-01-01,FOXA1 Induces E-Cadherin Expression at the Pro...,Biological & pharmaceutical bulletin,10.1248/bpb.b17-00307,Epithelial-to-mesenchymal transition (EMT) is ...,0.88,18,3.600000,NaN,NaN,15.0,"[""Antigens, CD"",""Breast Neoplasms"",""Cadherins""...","[""SFN protein, human"",""Hepatocyte Nuclear Fact...","{2019:5,2020:3,2021:1,2022:2}",Hepatocyte Nuclear Factor 3-alpha;Estrogen Rec...
1,23611783,2013-04-20,Identification and characterization of a novel...,Biochemical and biophysical research communica...,10.1016/j.bbrc.2013.04.022,"In plants, flavonoids have been shown to be su...",0.55,15,1.666670,NaN,NaN,17.0,"[""Amino Acid Sequence"",""Arabidopsis"",""Arabidop...","[""Differential quality"",""Characterization"",""Bo...","{2013:1,2014:1,2015:1,2018:1,2019:2,2022:1}",Kaempferols;Multigene Family;Arabidopsis Prote...
2,31440131,2019-08-07,The Verbal Interaction Social Threat Task: A N...,Frontiers in neuroscience,10.3389/fnins.2019.00830,"a priori In recent years, digital communicatio...",0.31,2,0.666667,NaN,NaN,NaN,NaN,"[""Neural"",""Societies"",""conversation"",""Moderate...",{2021:1},Kaempferols;Multigene Family;Arabidopsis Prote...
3,30551143,2019-02-28,Functionality and cross-regulation of the four...,Nucleic acids research,10.1093/nar/gky1256,Toxin-antitoxin (TA) systems are ubiquitous am...,1.37,14,4.666670,NaN,NaN,10.0,"[""Bacterial Proteins"",""Drug Resistance, Microb...","[""Antibiotic Resistance, Microbial"",""Cross-Tal...","{2019:3,2020:2,2021:5,2022:2}","Staphylococcal Infections;Drug Resistance, Mic..."
4,30229639,2018-09-18,Non-familial porphyria cutanea tarda: a rare d...,Giornale italiano di dermatologia e venereolog...,10.23736/S0392-0488.18.06052-2,NaN,0.00,0,0.000000,NaN,NaN,5.0,"[""Female"",""Humans"",""Middle Aged"",""Porphyria Cu...",NaN,NaN,Rare Diseases;Middle Aged;Humans;Female;Porphy...


In [18]:
max_retries = 3
retry_delay = 3

def heading_to_dui(heading):
    search_url = 'https://id.nlm.nih.gov/mesh/lookup/descriptor'
    params = {
        'label': heading,
        'match': 'exact',
        'year': 'current',
        'limit': 1,
    }
    for attempt in range(1, max_retries + 1):
        try:
            # Your code that may raise the exception
            response = requests.get(search_url, params=params)
            # Process the response or perform other operations
            result = response.json()
            dui = result[0]['resource'].split('/')[-1]
            return dui # Connection successful, exit the loop
        except requests.exceptions.ConnectionError as e:
            # Handling the specific exception
            print(f"Connection Error (attempt {attempt}):", e)
            if attempt < max_retries:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
        except Exception as e:
            # Handling other exceptions
            print("An error occurred:", e)
            break  # Exit the loop if other exception occurs

    raise TypeError(f"Failed to establish connection after {max_retries} attempts for {heading}.")

heading_to_dui('Arthroplasty, Replacement, Hip')

'D019644'

In [20]:
# gather all unique mesh topics by splitting the string by ; at each row, then return the unique values
distinct_topics = list(set(';'.join(pubmed_df['unique_mesh_topics'].tolist()).split(';')))

with open(PICKLES_ADDRESS + 'mesh_topics_list.pkl', 'wb') as handle:
    pickle.dump(distinct_topics, handle)
print('mesh_topics_list.pkl is saved.', len(distinct_topics))

mesh_topics_list.pkl is saved. 29749


In [19]:
# create a dictionary of mesh topics and their corresponding dui
heading2dui = {}
c = 0
for topic in tqdm(distinct_topics[:]):
    try:
        heading2dui[topic] = heading_to_dui(topic)
    except Exception as e:
        print('=' * 50)
        print('EXCEPT:', topic)
        print(e)
        print('=' * 50)
        
    if c % 1000 == 0:
        with open(PICKLES_ADDRESS + 'heading2dui.pkl', 'wb') as handle:
            pickle.dump(heading2dui, handle)
        print('heading2dui.pkl is saved.')
    c += 1

with open(PICKLES_ADDRESS + 'heading2dui.pkl', 'wb') as handle:
    pickle.dump(heading2dui, handle)
print('heading2dui.pkl is saved.', len(heading2dui))

  0%|          | 2/29749 [00:00<1:03:38,  7.79it/s]

heading2dui.pkl is saved.


  2%|▏         | 512/29749 [01:05<1:01:34,  7.91it/s]

Connection Error (attempt 1): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 3 seconds...


  3%|▎         | 949/29749 [02:04<1:01:17,  7.83it/s]

...

An error occurred: list index out of range
EXCEPT: Lycopersicon esculentum
Failed to establish connection after 3 attempts for Lycopersicon esculentum.


100%|█████████▉| 29745/29749 [1:06:02<00:00,  7.83it/s]

Connection Error (attempt 1): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying in 3 seconds...


100%|██████████| 29749/29749 [1:06:06<00:00,  7.50it/s]

heading2dui.pkl is saved. 29655


In [21]:
duis = []
for index, row in tqdm(pubmed_df.iterrows()):
    d = []
    for topic in row['unique_mesh_topics'].split(';'):
        try:
            d.append(heading2dui[topic])
        except:
            pass
    duis.append(';'.join(d))

pubmed_df['duis'] = duis
pubmed_df.head()

13517353it [07:04, 31841.92it/s]


,pmid,pub_date,pub_title,journal_title,doi,abstract,relative_citation_ratio,citation_count,citations_per_year,num_major_topics,major_topics,num_minor_topics,minor_topics,concept_list,yearly_citation_counts,unique_mesh_topics,duis
0,28867731,2017-01-01,FOXA1 Induces E-Cadherin Expression at the Pro...,Biological & pharmaceutical bulletin,10.1248/bpb.b17-00307,Epithelial-to-mesenchymal transition (EMT) is ...,0.88,18,3.600000,NaN,NaN,15.0,"[""Antigens, CD"",""Breast Neoplasms"",""Cadherins""...","[""SFN protein, human"",""Hepatocyte Nuclear Fact...","{2019:5,2020:3,2021:1,2022:2}",Hepatocyte Nuclear Factor 3-alpha;Estrogen Rec...,D051541;D047628;D010957;D000071250;D012333;D01...
1,23611783,2013-04-20,Identification and characterization of a novel...,Biochemical and biophysical research communica...,10.1016/j.bbrc.2013.04.022,"In plants, flavonoids have been shown to be su...",0.55,15,1.666670,NaN,NaN,17.0,"[""Amino Acid Sequence"",""Arabidopsis"",""Arabidop...","[""Differential quality"",""Characterization"",""Bo...","{2013:1,2014:1,2015:1,2018:1,2019:2,2022:1}",Kaempferols;Multigene Family;Arabidopsis Prote...,D044949;D005810;D029681;D005419;D017386;D01080...
2,31440131,2019-08-07,The Verbal Interaction Social Threat Task: A N...,Frontiers in neuroscience,10.3389/fnins.2019.00830,"a priori In recent years, digital communicatio...",0.31,2,0.666667,NaN,NaN,NaN,NaN,"[""Neural"",""Societies"",""conversation"",""Moderate...",{2021:1},Kaempferols;Multigene Family;Arabidopsis Prote...,D044949;D005810;D029681;D005419;D017386;D01080...
3,30551143,2019-02-28,Functionality and cross-regulation of the four...,Nucleic acids research,10.1093/nar/gky1256,Toxin-antitoxin (TA) systems are ubiquitous am...,1.37,14,4.666670,NaN,NaN,10.0,"[""Bacterial Proteins"",""Drug Resistance, Microb...","[""Antibiotic Resistance, Microbial"",""Cross-Tal...","{2019:3,2020:2,2021:5,2022:2}","Staphylococcal Infections;Drug Resistance, Mic...",D013203;D004352;D013211;D006801;D001426;D01596...
4,30229639,2018-09-18,Non-familial porphyria cutanea tarda: a rare d...,Giornale italiano di dermatologia e venereolog...,10.23736/S0392-0488.18.06052-2,NaN,0.00,0,0.000000,NaN,NaN,5.0,"[""Female"",""Humans"",""Middle Aged"",""Porphyria Cu...",NaN,NaN,Rare Diseases;Middle Aged;Humans;Female;Porphy...,D035583;D008875;D006801;D005260;D017119


In [22]:
pubmed_df.to_csv(PUBMED_ADDRESS + 'pubmed_dataset_with_duis.csv', index=False)
print('Saved')

Saved


In [23]:
# read the csv file
pubmed_df = pd.read_csv(PUBMED_ADDRESS + 'pubmed_dataset_with_duis.csv')
pubmed_df.head()

,pmid,pub_date,pub_title,journal_title,doi,abstract,relative_citation_ratio,citation_count,citations_per_year,num_major_topics,major_topics,num_minor_topics,minor_topics,concept_list,yearly_citation_counts,unique_mesh_topics,duis
0,28867731,2017-01-01,FOXA1 Induces E-Cadherin Expression at the Pro...,Biological & pharmaceutical bulletin,10.1248/bpb.b17-00307,Epithelial-to-mesenchymal transition (EMT) is ...,0.88,18,3.600000,NaN,NaN,15.0,"[""Antigens, CD"",""Breast Neoplasms"",""Cadherins""...","[""SFN protein, human"",""Hepatocyte Nuclear Fact...","{2019:5,2020:3,2021:1,2022:2}",Hepatocyte Nuclear Factor 3-alpha;Estrogen Rec...,D051541;D047628;D010957;D000071250;D012333;D01...
1,23611783,2013-04-20,Identification and characterization of a novel...,Biochemical and biophysical research communica...,10.1016/j.bbrc.2013.04.022,"In plants, flavonoids have been shown to be su...",0.55,15,1.666670,NaN,NaN,17.0,"[""Amino Acid Sequence"",""Arabidopsis"",""Arabidop...","[""Differential quality"",""Characterization"",""Bo...","{2013:1,2014:1,2015:1,2018:1,2019:2,2022:1}",Kaempferols;Multigene Family;Arabidopsis Prote...,D044949;D005810;D029681;D005419;D017386;D01080...
2,31440131,2019-08-07,The Verbal Interaction Social Threat Task: A N...,Frontiers in neuroscience,10.3389/fnins.2019.00830,"a priori In recent years, digital communicatio...",0.31,2,0.666667,NaN,NaN,NaN,NaN,"[""Neural"",""Societies"",""conversation"",""Moderate...",{2021:1},Kaempferols;Multigene Family;Arabidopsis Prote...,D044949;D005810;D029681;D005419;D017386;D01080...
3,30551143,2019-02-28,Functionality and cross-regulation of the four...,Nucleic acids research,10.1093/nar/gky1256,Toxin-antitoxin (TA) systems are ubiquitous am...,1.37,14,4.666670,NaN,NaN,10.0,"[""Bacterial Proteins"",""Drug Resistance, Microb...","[""Antibiotic Resistance, Microbial"",""Cross-Tal...","{2019:3,2020:2,2021:5,2022:2}","Staphylococcal Infections;Drug Resistance, Mic...",D013203;D004352;D013211;D006801;D001426;D01596...
4,30229639,2018-09-18,Non-familial porphyria cutanea tarda: a rare d...,Giornale italiano di dermatologia e venereolog...,10.23736/S0392-0488.18.06052-2,NaN,0.00,0,0.000000,NaN,NaN,5.0,"[""Female"",""Humans"",""Middle Aged"",""Porphyria Cu...",NaN,NaN,Rare Diseases;Middle Aged;Humans;Female;Porphy...,D035583;D008875;D006801;D005260;D017119
